In [5]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('all-MiniLM-L6-v2')

In [2]:
def get_average_semantic_similarity(sentences, model):
    #Compute embeddings
    embeddings = model.encode(sentences, convert_to_tensor=True)
    
    #Compute cosine-similarities for each sentence with each other sentence
    cosine_scores = util.cos_sim(embeddings, embeddings)
    # Extract upper/lower half
    upper_half = torch.triu(cosine_scores, diagonal=1)
    upper_half = upper_half[upper_half != 0]
#     upper_half = upper_half[upper_half != 1.0]
    average = torch.mean(upper_half)
    return average.item(), cosine_scores

In [3]:
import json
import uuid
def read_eval_results(results):
    with open(results, 'r') as json_file:
        json_list_str = list(json_file)
    json_data = dict()
    json_list = list()
    for json_str in json_list_str:
        result = json.loads(json_str)
        json_data[str(uuid.uuid4())] = result
        json_list.append(result)
    return json_list

In [4]:
def get_all_generations_from_json_data(json_list):
    all_generations = []
    for item in data:
        try:
            all_generations.append(item['generation'][0])
        except:
            print(item['generation'])
            all_generations.append([])
    return all_generations

In [79]:
# all_generations = []
# import pickle
# with open("/Users/sankethrangreji/Coursework/11797 Question Answering/QA11797/LLaMA-2 70b results/llama_text_gsmhard_processed_generations.pkl", "rb") as f:
#     all_generations = pickle.load(f)

In [6]:
import pickle
def read_pickle_generations(filepath):
    all_generations = []
    with open(filepath, "rb") as f:
        all_generations = pickle.load(f)
    return all_generations

In [7]:
data = read_eval_results("/Users/sankethrangreji/Coursework/11797 Question Answering/QA11797/openai code generations/gsm_code_text-davinci-003_t_1_k_10.jsonl")

In [82]:
# data

In [8]:
all_generations = []
for item in data:
    try:
        all_generations.append(item['generation'][0])
    except:
        print(item['generation'])
        all_generations.append([])

In [9]:
all_generations = read_pickle_generations("/Users/sankethrangreji/Coursework/11797 Question Answering/QA11797/temperature_experiments/1_0/Code Generations/vllm_llama2_70b_1_object_counting_code_generation.pkl")

In [10]:
len(all_generations)

250

In [67]:
# all_generations[34]

In [11]:
from tqdm import tqdm
import numpy as np

In [12]:
def get_avg_similarity_score_all_generations(model, all_generations):
    avg_similarity_scores = []
    for generations in tqdm(all_generations):
        if generations:
            avg, cosine_scores = get_average_semantic_similarity(generations, model)
            avg_similarity_scores.append(avg)
    return np.mean(avg_similarity_scores)

### Uncomment and run below cell for obtainining similarity score for a single file read above

In [13]:
# avg_similarity_scores = []
# for generations in tqdm(all_generations):
#     if generations:
#         avg, cosine_scores = get_average_semantic_similarity(generations, model)
#         avg_similarity_scores.append(avg)
        

In [14]:
# np.mean(avg_similarity_scores)

In [15]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

### Run below cells to generate similarity scores for all result or pickle files in the given folder(to be provided as FILE_PATH)

In [19]:
import glob

In [20]:
FILE_PATH = "/Users/sankethrangreji/Coursework/11797 Question Answering/QA11797/temperature_experiments_13b/0_7/Text generations/*.pkl"

In [30]:
import glob
pattern = FILE_PATH

# Use glob to get a list of files that match the pattern
file_list = glob.glob(pattern)

# Now, file_list contains the paths of all .pkl files in the current directory
# You can loop through this list to process these files or perform any other operations.
for file_path in file_list:
    all_generations = []
    try:
        if file_path.endswith('pkl'):
            all_generations = read_pickle_generations(file_path)
#             print(all_generations[0])
        elif file_path.endswith('jsonl'):
            data = read_eval_results(file_path)
            all_generations = get_all_generations_from_json_data(data)
#             print(all_generations[0])
        else:
            print("Invalid file type {}".format(file_path.split("/")[-1].split(".")[-1]))
            continue
        if all_generations:
            score = get_avg_similarity_score_all_generations(model, all_generations)
        print("Score for {} is : {}".format(file_path.split('\n')[-1], score))
    except Exception as e:
        print("Error : ", e)

100%|██████████████████████████████████████████████████████████████████████████████| 32/32 [00:03<00:00,  9.59it/s]


Score for /Users/sankethrangreji/Coursework/11797 Question Answering/QA11797/temperature_experiments_13b/0_7/Text generations/vllm_llama2_13b_0.7_repeat_copy_text_generation.pkl is : 0.9507282245904207


100%|██████████████████████████████████████████████████████████████████████████| 1319/1319 [03:23<00:00,  6.47it/s]


Score for /Users/sankethrangreji/Coursework/11797 Question Answering/QA11797/temperature_experiments_13b/0_7/Text generations/vllm_llama2_13b_0.7_gsmhardv2_text_generation.pkl is : 0.8348019924283118


100%|████████████████████████████████████████████████████████████████████████████| 369/369 [00:27<00:00, 13.25it/s]


Score for /Users/sankethrangreji/Coursework/11797 Question Answering/QA11797/temperature_experiments_13b/0_7/Text generations/vllm_llama2_13b_0.7_date_understanding_text_generation.pkl is : 0.8681171933809916


100%|██████████████████████████████████████████████████████████████████████████| 1319/1319 [02:40<00:00,  8.20it/s]


Score for /Users/sankethrangreji/Coursework/11797 Question Answering/QA11797/temperature_experiments_13b/0_7/Text generations/vllm_llama2_13b_0.7_gsm_text_generation.pkl is : 0.8540954859849628


100%|████████████████████████████████████████████████████████████████████████████| 250/250 [00:14<00:00, 16.93it/s]

Score for /Users/sankethrangreji/Coursework/11797 Question Answering/QA11797/temperature_experiments_13b/0_7/Text generations/vllm_llama2_13b_0.7_object_counting_text_generation.pkl is : 0.9865523309707641
